# Analyzing CIA Factbook Data Using SQL

### In this project, we will be analyzing data from the CIA World Factbook,  a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:

* population — the global population.
* population_growth — the annual population growth rate, as a percentage.
* area — the total land and water area.

## Connecting the database file and data overview

In [12]:
!conda install -yc conda-forge ipython-sql

Solving environment: ...working... done

## Package Plan ##

  environment location: G:\anaconda3

  added / updated specs:
    - ipython-sql


The following packages will be SUPERSEDED by a higher-priority channel:

  conda               pkgs/main::conda-4.9.2-py37haa95532_0 --> conda-forge::conda-4.9.2-py37h03978a9_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [27]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

#### To query the database to get this information directly:

In [8]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


#### First 5 rows of the 'facts' table in the database

In [11]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


#### Here are the descriptions for some of the columns:

* name — the name of the country.
* area— the country's total area (both land and water).
* area_land — the country's land area in square kilometers.
* area_water — the country's waterarea in square kilometers.
* population — the country's population.
* population_growth— the country's population growth as a percentage.
* birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
* death_rate — the country's death rate, or the number of death per year per 1,000 people.

## Calculating some summary statistics and looking for any outlier countries

In [4]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


In [5]:
%%sql
SELECT name, population
  FROM facts
 WHERE population IN (SELECT MIN(population)
                                      FROM facts
                                    );

 * sqlite:///factbook.db
Done.


name,population
Antarctica,0


%%sql
SELECT name, population
  FROM facts
 WHERE population IN (SELECT MAX(population)
                                      FROM facts
                                    );

#### We can see that the data includes a row for the whole World called 'World' and that Antarctica has a population of 0 which are both skewing our country population data. We'll rerun the same stats without these rows below.

In [17]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts
 WHERE name <> 'World'
  AND name <> 'Antarctica';

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
48,1367485388,0.0,4.02


In [18]:
%%sql
SELECT name, population
  FROM facts
 WHERE population IN (SELECT MIN(population) FROM facts
                       WHERE name <> 'World'
                         AND name <> 'Antarctica');

 * sqlite:///factbook.db
Done.


name,population
Pitcairn Islands,48


#### From the above query, we can see that the British territory of Pitcairn Islands has the lowest population of any territory with 48 people, but it is technically not a country. In order to exclude Island territories and cities, we excluded them in our SQL query filter and returned Niue as the Country with the lowest population: 1190.

In [20]:
%%sql
SELECT name, population
  FROM facts
 WHERE population IN (SELECT MIN(population) FROM facts
                       WHERE name <> 'World'
                         AND name <> 'Antarctica'
                         AND name NOT LIKE '% Islands'
                         AND name NOT LIKE '% City)');

 * sqlite:///factbook.db
Done.


name,population
Niue,1190


#### Below we can see there are 14 countries that have dense populations in below average areas.

In [25]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                                         FROM facts
                                      WHERE name <> 'World'
                         AND name <> 'Antarctica')
  AND area < (SELECT AVG(area)
                         FROM facts
                        WHERE name <> 'World'
                         AND name <> 'Antarctica')
 AND name <> 'World'
ORDER BY population DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


## Further questions

### We will answer the following questions below:
* Which country has the most people? Which country has the highest growth rate?
* Which countries have the highest ratios of water to land? Which countries have more water than land?
* Which countries will add the most people to their populations next year?
* Which countries have a higher death rate than birth rate?
* Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?

#### The country with the most people is China.

In [33]:
%%sql
SELECT *
  FROM facts
 WHERE name <> 'World'
  AND name <> 'Antarctica'
  AND name NOT LIKE '% Island'
  AND name NOT LIKE '% Islands'
  AND name NOT LIKE '% City'
 ORDER BY population DESC
 Limit 1;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


#### The country has the highest growth rate is South Sudan.

In [34]:
%%sql
SELECT *
  FROM facts
 WHERE name <> 'World'
  AND name <> 'Antarctica'
  AND name NOT LIKE '% Island'
  AND name NOT LIKE '% Islands'
  AND name NOT LIKE '% City'
 ORDER BY population_growth DESC
 Limit 1;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


#### The countries with the highest ratios of water to land is British Indian Ocean Territory, but it is not technically an autonomous country.

In [41]:
%%sql
SELECT *, CAST(area_water AS FLOAT) / area_land AS water_land_ratio
  FROM facts
 WHERE name <> 'World'
  AND name <> 'Antarctica'
  AND name NOT LIKE '% Islands'
  AND name NOT LIKE '% City'
 ORDER BY water_land_ratio DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,water_land_ratio
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None,905.6666666666666
246,rq,Puerto Rico,13791,8870,4921,3598357,0.6,10.86,8.67,8.15,0.5547914317925592
12,bf,"Bahamas, The",13880,10010,3870,324597,0.85,15.5,7.05,0.0,0.3866133866133866
71,pu,Guinea-Bissau,36125,28120,8005,1726170,1.91,33.38,14.33,0.0,0.2846728307254623
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0,0.25939625850340137
125,nl,Netherlands,41543,33893,7650,16947904,0.41,10.83,8.66,1.95,0.22571032366565366
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74,0.22292237442922375
56,er,Eritrea,117600,101000,16600,6527689,2.25,30.0,7.52,0.0,0.16435643564356436
99,li,Liberia,111369,96320,15049,4195666,2.47,34.41,9.69,0.0,0.15623961794019933
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46,0.1405085657217485


#### We exclude this territory and rerun the code. The countries with the highest ratios of water to land is Puerto Rico.

In [40]:
%%sql
SELECT *, CAST(area_water AS FLOAT) / area_land AS water_land_ratio
  FROM facts
 WHERE name <> 'World'
  AND name <> 'Antarctica'
  AND name NOT LIKE '% Islands'
  AND name NOT LIKE '% City'
  AND name NOT LIKE '% Territory'
 ORDER BY water_land_ratio DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,water_land_ratio
246,rq,Puerto Rico,13791,8870,4921,3598357,0.6,10.86,8.67,8.15,0.5547914317925592
12,bf,"Bahamas, The",13880,10010,3870,324597,0.85,15.5,7.05,0.0,0.3866133866133866
71,pu,Guinea-Bissau,36125,28120,8005,1726170,1.91,33.38,14.33,0.0,0.2846728307254623
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0,0.25939625850340137
125,nl,Netherlands,41543,33893,7650,16947904,0.41,10.83,8.66,1.95,0.22571032366565366
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74,0.22292237442922375
56,er,Eritrea,117600,101000,16600,6527689,2.25,30.0,7.52,0.0,0.16435643564356436
99,li,Liberia,111369,96320,15049,4195666,2.47,34.41,9.69,0.0,0.15623961794019933
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46,0.1405085657217485
63,ga,"Gambia, The",11300,10120,1180,1967709,2.16,30.86,7.15,2.12,0.116600790513834


#### The country with more water than land is Puerto Rico.

In [49]:
%%sql
SELECT *, CAST(area_water AS FLOAT) / area_land AS water_land_ratio
  FROM facts
 WHERE name <> 'World'
  AND name <> 'Antarctica'
  AND name NOT LIKE '% Islands'
  AND name NOT LIKE '% City'
  AND name NOT LIKE '% Territory'
  AND water_land_ratio > 0.5
 ORDER BY water_land_ratio DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,water_land_ratio
246,rq,Puerto Rico,13791,8870,4921,3598357,0.6,10.86,8.67,8.15,0.5547914317925592


#### Countries adding the most people to their populations next year are:

In [51]:
%%sql
SELECT *, ROUND(birth_rate * population) AS growth
  FROM facts
 WHERE name <> 'World'
  AND name <> 'Antarctica'
  AND name NOT LIKE '% Islands'
  AND name NOT LIKE '% City'
  AND name NOT LIKE '% Territory'
 ORDER BY growth DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,growth
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04,24470648667.0
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44,17079892496.0
129,ni,Nigeria,923768,910768,13000,181562056,2.45,37.64,12.9,0.22,6833995788.0
197,ee,European Union,4324782,None,None,513949445,0.25,10.2,10.2,2.5,5242284339.0
132,pk,Pakistan,796095,770875,25220,199085847,1.46,22.58,6.49,1.54,4495358425.0
78,id,Indonesia,1904569,1811569,93000,255993674,0.92,16.72,6.37,1.16,4280214229.0
186,us,United States,9826675,9161966,664709,321368864,0.78,12.49,8.15,3.86,4013897111.0
58,et,Ethiopia,1104300,None,104300,99465819,2.89,37.27,8.19,0.22,3707091074.0
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46,3571766729.0
24,br,Brazil,8515770,8358140,157630,204259812,0.77,14.46,6.58,0.14,2953596882.0


#### Countries have a higher death rate than birth rate are:

In [54]:
%%sql
SELECT *
  FROM facts
 WHERE name <> 'World'
  AND name <> 'Antarctica'
  AND name NOT LIKE '% Islands'
  AND name NOT LIKE '% City'
  AND name NOT LIKE '% Territory'
  AND death_rate > birth_rate
 ORDER BY id;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7
22,bk,Bosnia and Herzegovina,51197,51187,10,3867055,0.13,8.87,9.75,0.38
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
44,hr,Croatia,56594,55974,620,4464844,0.13,9.45,12.18,1.39
47,ez,Czech Republic,78867,77247,1620,10644842,0.16,9.63,10.34,2.33
57,en,Estonia,45228,42388,2840,1265420,0.55,10.51,12.4,3.6
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
67,gr,Greece,131957,130647,1310,10775643,0.01,8.66,11.09,2.32
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33


#### Countries with the highest population/area ratio are:

In [58]:
%%sql
SELECT *, CAST(population AS FLOAT) / area AS population_area_ratio
  FROM facts
 WHERE name <> 'World'
  AND name <> 'Antarctica'
  AND name NOT LIKE '% Islands'
  AND name NOT LIKE '% City'
  AND name NOT LIKE '% Territory'
 ORDER BY population_area_ratio DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,population_area_ratio
205,mc,Macau,28,28,0,592731,0.8,8.88,4.22,3.37,21168.964285714286
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83,15267.5
156,sn,Singapore,697,687,10,5674472,1.89,8.27,3.43,14.05,8141.279770444763
204,hk,Hong Kong,1108,1073,35,7141106,0.38,9.23,7.07,1.68,6445.041516245487
251,gz,Gaza Strip,360,360,0,1869055,2.81,31.11,3.04,0.0,5191.819444444444
233,gi,Gibraltar,6,6,0,29258,0.24,14.08,8.37,3.28,4876.333333333333
13,ba,Bahrain,760,760,0,1346613,2.41,13.66,2.69,13.09,1771.8592105263158
108,mv,Maldives,298,298,0,393253,0.08,15.75,3.89,12.68,1319.6409395973155
110,mt,Malta,316,316,0,413965,0.31,10.18,9.09,1.98,1310.01582278481
227,bd,Bermuda,54,54,0,70196,0.5,11.33,8.23,1.88,1299.9259259259259
